In [31]:
import pandas as pd
from sqlalchemy import create_engine
import psycopg2
import urllib.parse
from urllib.parse import quote_plus as urlquote


In [39]:
blush_csv_path = "blush_fixed.csv"
eyeshadow_csv_path = "eyeshadow_fixed.csv"
lipstick_csv_path = "lipstick_fixed.csv"
foundation_csv_path = "foundation_fixed.csv"

blush_df = pd.read_csv(blush_csv_path)
eyeshadow_df = pd.read_csv(eyeshadow_csv_path)
lipstick_df = pd.read_csv(lipstick_csv_path)
foundation_df = pd.read_csv(foundation_csv_path)

Blush_eyeshadow_df = pd.merge(blush_df,eyeshadow_df, how='outer')
Blush_eyeshadow_lipsick_df = pd.merge(Blush_eyeshadow_df, lipstick_df, how='outer')
Ulta_Products_df = pd.merge(Blush_eyeshadow_lipsick_df, foundation_df, how='outer')
Ulta_Products_df

,Brand,Product,Product Type,Price,Rating,Image URL
0,KYLIE COSMETICS,Blush,Blush,18.00,3.8,https://images.ulta.com/is/image/Ulta/2543053?...
1,NARS,Blush,Blush,30.00,4.6,https://images.ulta.com/is/image/Ulta/2503561?...
2,Tarte,Amazonian Clay 12 Hour Blush,Blush,29.00,4.6,https://images.ulta.com/is/image/Ulta/2230172?...
3,Smashbox,Halo Cheek Palette,Blush,35.00,4.2,https://images.ulta.com/is/image/Ulta/2569811?...
4,Milani,Baked Blush,Blush,9.49,4.6,https://images.ulta.com/is/image/Ulta/2519994?...
...,...,...,...,...,...,...
378,L'Oréal,Infallible Pro-Glow Foundation,Foundation,12.99,4.2,https://images.ulta.com/is/image/Ulta/2305296?...
379,NYX Professional Makeup,Can't Stop Won't Stop Foundation,Foundation,15.00,3.7,https://images.ulta.com/is/image/Ulta/2531965?...
380,ULTA,Youthful Glow Foundation Serum Drops,Foundation,18.00,3.6,https://images.ulta.com/is/image/Ulta/2534921?...
381,Tarte,Travel Size Babassu Foundcealer Skincare Found...,Foundation,15.00,4.7,https://images.ulta.com/is/image/Ulta/2557012?...


In [40]:
Ulta_products_cols = ["Brand","Product", "Product Type", "Price", "Rating", "Image URL"]
Ulta_Products_transformed = Ulta_Products_df[Ulta_products_cols].copy()

Ulta_Products_transformed = Ulta_Products_transformed.rename(columns= {"Brand" : "brand",
                                                          "Product" : "product", 
                                                          "Product Type" : "product_type", 
                                                          "Price" :      "price", 
                                                          "Rating" :   "rating", 
                                                          "Image URL" :     "image_url"}) 
Ulta_Products_transformed.head()

,brand,product,product_type,price,rating,image_url
0,KYLIE COSMETICS,Blush,Blush,18.00,3.8,https://images.ulta.com/is/image/Ulta/2543053?...
1,NARS,Blush,Blush,30.00,4.6,https://images.ulta.com/is/image/Ulta/2503561?...
2,Tarte,Amazonian Clay 12 Hour Blush,Blush,29.00,4.6,https://images.ulta.com/is/image/Ulta/2230172?...
3,Smashbox,Halo Cheek Palette,Blush,35.00,4.2,https://images.ulta.com/is/image/Ulta/2569811?...
4,Milani,Baked Blush,Blush,9.49,4.6,https://images.ulta.com/is/image/Ulta/2519994?...


In [5]:
#connection_string = "postgres:@localhost:5432/ulta_db"
#engine = create_engine(f'postgresql://{connection_string}')


In [6]:
#engine.table_names()

['ulta']

In [17]:
#Ulta_Products_transformed.to_sql(name='ulta', con=engine, if_exists='append', index= False)

In [41]:
ulta_dictionary = Ulta_Products_transformed.to_dict(orient='records')

In [42]:
import pymongo

# Setup connection to mongodb
conn = "mongodb://localhost:27017"
client = pymongo.MongoClient(conn)

# Select database and collection to use
db = client.ulta_db
ulta = db.ulta

In [43]:
ulta.insert_many(ulta_dictionary)